#  EXERCICE1

In [1]:
from gurobipy import *
import pandas as pd 
import numpy as np
from pandas import DataFrame, read_csv


In [2]:
file = r'villes.xlsx'
df = pd.read_excel(file)



In [3]:
Villes = df['Ville']
population = df['Population'].tolist()
Distances = df.iloc[:,2:].to_numpy()


In [4]:
nb_villes = len(Villes)
k_index = [0, 1 , 2]
k = len(k_index)
alpha = 0.1
population_max = sum(population)*(1 + alpha)/k


In [5]:
m = Model('Approach One')
m.setParam('OutputFlag', False) 

Academic license - for non-commercial use only - expires 2021-02-20
Using license file /Users/youcefmcirdi/gurobi.lic


In [6]:
x1 = np.empty((nb_villes,k), dtype=Var)

for i in range(nb_villes):
    for j in range(k):
        x1[i,j] = m.addVar(vtype=GRB.BINARY, lb=0, name="x(%s/%s)" % (Villes[i],Villes[k_index[j]]))
    
m.update()



In [7]:
obj = LinExpr();
obj = 0
for i in range(nb_villes):
    for j in range(k):
        obj += population[i] * Distances[i,k_index[j]] * x1[i,j]
        
obj /= sum(population)



In [8]:
m.setObjective(obj,GRB.MINIMIZE)   

In [9]:
for i in range(nb_villes):
        m.addConstr(quicksum(x1[i,:]) == 1)
        

for i in range(nb_villes):
        m.addConstr(quicksum([population[i]*x1[i,j] for j in range(k)]) <= population_max)
    


In [10]:
m.optimize()

In [11]:
result1 = {}
for i in range(nb_villes):
    for j in range(k):
        if x1[i,j].X == 1:
            result1[Villes[i]] = Villes[k_index[j]]
result1 = pd.DataFrame.from_dict(result1,orient='index',columns=['Unité de soin'])




In [12]:
print('Solution optimale pour alpha = 0.1 et k = 3:\n',result1)

print('Valeur de la fonction objectif :\n', m.objVal)




Solution optimale pour alpha = 0.1 et k = 3:
               Unité de soin
Toulouse           Toulouse
Nice                   Nice
Nantes               Nantes
Montpellier        Toulouse
Strasbourg             Nice
Bordeaux           Toulouse
Lille                Nantes
Rennes               Nantes
Reims                Nantes
Saint-Étienne          Nice
Toulon                 Nice
Le Havre             Nantes
Grenoble               Nice
Dijon                Nantes
Angers               Nantes
Valeur de la fonction objectif :
 272.74793968298496


# EXERCICE  2 

In [57]:
file = r'villes.xlsx'
df = pd.read_excel(file)



Villes = df['Ville']
population = df['Population'].tolist()
Distances = df.iloc[:,2:].to_numpy()



nb_villes = len(Villes)
k_index = [0, 1, 9, 12]
k = len(k_index)
alpha = 0.2
population_max = sum(population)*(1 + alpha)/k



In [58]:
m2 = Model('Approach two')
m2.setParam('OutputFlag', False) 

In [59]:
x2 = np.empty((nb_villes,nb_villes), dtype=Var)

for i in range(nb_villes):
    for j in range(nb_villes):
        x2[i,j] = m2.addVar(vtype=GRB.BINARY, lb=0, name="x2(%s/%s)" % (Villes[i],Villes[j]))
    


In [60]:
z = []        
for i in range(nb_villes):
    z.append(m2.addVar(vtype=GRB.BINARY, lb=0, name="z%d" % (i+1)))

m2.update()
    

In [61]:
obj = LinExpr();
obj = 0
for i in range(nb_villes):
    for j in range(nb_villes):
        obj += population[i] * Distances[i,j] * x2[i,j] 
obj /= sum(population)





In [62]:
m2.setObjective(obj,GRB.MINIMIZE)   

In [63]:
m2.addConstr(sum(z) == k)

for j in range(nb_villes):
    m2.addConstr(quicksum([population[i]*x2[i,j] for i in range(nb_villes)]) <= z[j]*population_max)
for i in range(nb_villes):
    m2.addConstr(quicksum(x2[i,:]) == 1)




In [64]:
for j in range(nb_villes):
    m2.addConstr(quicksum([population[i]*x2[i,j] for i in range(nb_villes)]) <= z[j]*population_max)


In [65]:
for i in range(nb_villes):
    m2.addConstr(quicksum(x2[i,:]) == 1)

m2.optimize()   



In [66]:
K_2 = [i for i in range(len(z)) if z[i].x == 1]
result2 = {}

In [67]:
for i in range(nb_villes):
    for j in range(k):
        if x2[i,K_2[j]].x == 1:
            result2[Villes[i]] = Villes[K_2[j]]
result2 = pd.DataFrame.from_dict(result2,orient='index',columns=['Unités de soins'])

In [68]:
print('Solution optimale pour alpha = 0.2 et k = 4\n',result2)
print('Valeur de la fonction objectif de model 2:\n', m2.objVal)




Solution optimale pour alpha = 0.2 et k = 4
               Unités de soins
Toulouse             Toulouse
Nice                   Toulon
Nantes                 Nantes
Montpellier          Toulouse
Strasbourg              Reims
Bordeaux             Toulouse
Lille                   Reims
Rennes                 Nantes
Reims                   Reims
Saint-Étienne          Toulon
Toulon                 Toulon
Le Havre                Reims
Grenoble               Toulon
Dijon                   Reims
Angers                 Nantes
Valeur de la fonction objectif de model 2:
 167.035101977837


In [69]:
m2_2 = Model('Approach two')
m2_2.setParam('OutputFlag', False) 

In [70]:
x2_2 = np.empty((nb_villes,nb_villes), dtype=Var)

for i in range(nb_villes):
    for j in range(nb_villes):
        x2_2[i,j] = m2_2.addVar(vtype=GRB.BINARY, lb=0, name="x2_2(%s/%s)" % (Villes[i],Villes[j]))
    



In [71]:
z_2 = []        
for i in range(nb_villes):
    z_2.append(m2_2.addVar(vtype=GRB.BINARY, lb=0, name="z_2%d" % (i+1)))

m2_2.update()
    

In [72]:
D_M = m2_2.addVar(vtype=GRB.CONTINUOUS,lb=0) 
m2_2.update()




In [73]:
obj = LinExpr();
obj = D_M

m2_2.setObjective(obj,GRB.MINIMIZE)   
    

In [74]:
m2_2.addConstr(sum(z_2) == k)




<gurobi.Constr *Awaiting Model Update*>

In [75]:

for j in range(nb_villes):
    m2_2.addConstr(quicksum([population[i]*x2_2[i,j] for i in range(nb_villes)]) <= z_2[j]*population_max)
    
    
for i in range(nb_villes):
    m2_2.addConstr(quicksum(x2_2[i,:]) == 1)


for i in range(nb_villes):
    d_max = 0
    for j in range(nb_villes):
            d_max += x2_2[i,j]*Distances[i,j]
    m2_2.addConstr(d_max <= D_M )



In [76]:
m2_2.optimize()


In [77]:
K_2_2 = [i for i in range(len(z_2)) if z_2[i].x == 1]
result2_2 = {}
for i in range(nb_villes):
    for j in range(k):
            if x2_2[i,K_2_2[j]].x == 1:
                result2_2[Villes[i]] = Villes[K_2_2[j]]
result2_2 = pd.DataFrame.from_dict(result2_2,orient='index',columns=['Unités de soins'])  



In [80]:
print('Solution optimale:\n',result2_2)
print('Valeur de la fonction objectif de model 2-2 pour alpha = 0.2 et k = :\n', m2_2.objVal)




Solution optimale:
               Unités de soins
Toulouse             Toulouse
Nice              Montpellier
Nantes                 Angers
Montpellier          Toulouse
Strasbourg              Reims
Bordeaux             Toulouse
Lille                   Reims
Rennes                 Angers
Reims                   Reims
Saint-Étienne     Montpellier
Toulon            Montpellier
Le Havre               Angers
Grenoble          Montpellier
Dijon                   Reims
Angers                 Angers
Valeur de la fonction objectif de model 2-2 pour alpha = 0.2 et k = :
 347.0


# Exercice 3 

In [81]:
m3= Model('exo3')
m3.setParam('OutputFlag', False) 

In [82]:
units = [5, 6, 10, 13, 14]
p1 = [150,150,25,150,25]
p2 = [100,25,45,150,180]
k = len(units)

In [83]:
# M de coût  
Cij = np.zeros((k,k))
for i in range(k):
    for j in range(k):
        Cij[i,j] = Distances[units[i],units[j]]

In [84]:
#var de decision
x3 = np.empty((k,k), dtype=Var)
for i in range(k):
    for j in range(k):
        x3[i,j] = m3.addVar(vtype=GRB.INTEGER, lb=0, name="x3(%d/%d)" % (i,j))
    

    
m3.update()

In [85]:
obj =  LinExpr();
obj = 0  


In [86]:
#func obj
for i in range(k):
    for j in range(k):
        obj += Cij[i,j] * x3[i,j]
            
m3.setObjective(obj,GRB.MINIMIZE) 
    

In [ ]:
for i in range(k):
    m3.addConstr(quicksum(x3[i,:]) == p1[i])
for j in range(k):
    m3.addConstr(quicksum(x3[:,j]) <= 100)
    

In [ ]:
m3.optimize()



In [ ]:
M = np.zeros((k+1,k+1),dtype=int)
for i in range(k):
    for j in range(k):
        M[i,j] = x3[i,j].x
for i in range(k):
    M[i,k] = p1[i]
for j in range(k):
    M[k,j] = 100
M[k,k] = sum(p1)

M = pd.DataFrame(M,index=list(Villes[units])+['Offre'],columns=list(Villes[units])+['Demande'])

In [ ]:
print('Solution optimale pour le P1\n: ',M)
print('Valeur de la fonction objectif P1:', m3.objVal)

In [87]:
for i in range(k):
    m3.addConstr(quicksum(x3[i,:]) == p2[i])
for j in range(k):
    m3.addConstr(quicksum(x3[:,j]) <= 100)
m3.optimize()

   

In [88]:
M = np.zeros((k+1,k+1),dtype=int)
for i in range(k):
    for j in range(k):
        M[i,j] = x3[i,j].x
for i in range(k):
    M[i,k] = p2[i]
for j in range(k):
    M[k,j] = 100
M[k,k] = sum(p2)

M = pd.DataFrame(M,index=list(Villes[units])+['Offre'],columns=list(Villes[units])+['Demande'])

In [89]:
print('Solution optimale pour le P1\n: ',M)
print('Valeur de la fonction objectif P2:', m3.objVal)

Solution optimale pour le P1
:            Bordeaux  Lille  Toulon  Dijon  Angers  Demande
Bordeaux       100      0       0      0       0      100
Lille            0     25       0      0       0       25
Toulon           0      0      45      0       0       45
Dijon            0      0      50    100       0      150
Angers           0     75       5      0     100      180
Offre          100    100     100    100     100      500
Valeur de la fonction objectif P2: 71520.0
